# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use DataFrame to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!

In [1]:
import string
import re, collections
import ast

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

## Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

**index_col=False** make sure that the row number of the csv file is **not** used as index.

As a result, index label will be default to 0, 1, ... and could be accessed by .loc[number], which is important later on when splitting training and test data.

In [4]:
products = pd.read_csv("./data/amazon_baby.csv", index_col=False)

Now, let us see a preview of what the dataset looks like.

In [5]:
products.head(5)

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


Some reviews are obviously missing (**NaN**). Replace **NaN** reviews with " " to avoid errors when processing texts later on.

In [6]:
products["review"].fillna(" ",inplace=True)

## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [7]:
products.iloc[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [8]:
def remove_punctuation(text):
    # str.maketrans create a translation table
    # The translation table is built from a dictionary using a comprehension
    # this maps every character from string.punctuation to None
    translator = str.maketrans({key: None for key in string.punctuation})
    return text.translate(translator)

In [9]:
review_without_punctuation = products["review"].apply(remove_punctuation)

In [10]:
reviews = review_without_punctuation.values.tolist()

### My own bag of words. To later compare with scikit-learn
Below may not be the best word counters for BAG of WORD. There are indeed (significant??) difference with scikit-learn vectorizer

In [11]:
%%time
bagofwords = pd.DataFrame((str(dict(collections.Counter(re.findall(r"\w+", txt)))) for txt in reviews),columns=["word_count"])

Wall time: 9.74 s


Insert **["review_clean"]** and **["word_count"]** columns in DataFrame **products**

In [12]:
products["review_clean"] = review_without_punctuation
products.insert(len(products.columns),"word_count",bagofwords)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [13]:
products.iloc[269]["word_count"]

"{'A': 1, 'house': 1, 'in': 1, 'favorite': 1, 'our': 1}"

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [14]:
products = products[products["rating"] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [15]:
products["sentiment"] = products["rating"].apply(lambda rating : +1 if rating > 3 else -1)

**Reset index as the training and test sets make use of reset indexes !!**

In [16]:
products.reset_index(drop=True, inplace=True)

In [17]:
products.head(5)

,name,review,rating,review_clean,word_count,sentiment
0,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{'i': 1, 'not': 2, 'early': 1, 'and': 3, 'keps...",1
1,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"{'quilt': 1, 'soft': 1, 'Very': 1, 'anyone': 1...",1
2,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"{'well': 1, 'else': 1, 'getting': 1, 'What': 1...",1
3,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"{'pacifier': 2, 'their': 2, 'help': 1, 'an': 1...",1
4,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"{'pacifier': 2, 'their': 1, 'When': 1, 'recomm...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Load training and test sets

Load the **training set** from file provided by the course.

In [18]:
f = open("./data/module-2-assignment-train-idx.json")
train_data_index = f.readline()
f.close()

In [19]:
# Transform the read string into a list
train_data_index = ast.literal_eval(train_data_index)

In [20]:
train_data = products.loc[train_data_index]

Load the **test set** from file provided by the course.

In [21]:
f = open("./data/module-2-assignment-test-idx.json")
test_data_index = f.readline()
f.close()

In [22]:
# Transform the read string into a list
test_data_index = ast.literal_eval(test_data_index)

In [23]:
test_data = products.loc[test_data_index]

In [24]:
print("train_data shape:",train_data.shape)
print("test_data length:",test_data.shape)

train_data shape: (133416, 6)
test_data length: (33336, 6)


# Train a sentiment classifier with logistic regression

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, **scikit-learn** and many other tools use sparse matrices to store a collection of word count vectors.
Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:
* Learn a vocabulary (set of all words) from the **training data**. Only the words that show up in the training data will be considered for feature extraction.
* Compute the occurrences of the words in each review and collect them into a row vector.
* Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix **train_matrix**.
* Using the **same mapping** between words and columns, **convert the test** data into a sparse matrix **test_matrix**.

## Build the word count vector for each review with scikit-learn

### Utility function to convert a sparse vector to a word count
**sparse_vector_to_word_count** returns a word_count from a sparse vector (e.g. one row of the vectorizer.fit_transform() run on all reviews).

In [25]:
def sparse_vector_to_word_count(vocabulary,sparse_vector):
    wc_dict = dict(zip(vocabulary,np.asarray(sparse_vector.sum(axis=0)).ravel()))
    # Remove all words with count = 0
    wc_dict = { k:v for k,v in wc_dict.items() if v!=0 }
    return(wc_dict)

### Learn vocabulary on training data

In [26]:
# Notice the token_pattern argument in the constructor. This is to keep single-letter "words"
vectorizer = CountVectorizer(analyzer = "word",tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             token_pattern=r'\b\w+\b')

First, learn vocabulary from the **training data** and assign columns to words and convert the training data into a sparse matrix.

In [27]:
%%time
train_matrix = vectorizer.fit_transform(train_data["review_clean"])

Wall time: 7.7 s


In [28]:
# Number of words that have been ignored.
# the list of words ignored could be accessed by attribute stop_words_
print("number of stop_words:",len(vectorizer.stop_words_))

number of stop_words: 0


In [29]:
print("vocabulary size:",len(vectorizer.vocabulary_))

vocabulary size: 121712


Then convert the **test** data into a sparse matrix using the same word‐column mapping.

In [30]:
%%time
test_matrix = vectorizer.transform(test_data['review_clean'])

Wall time: 1.89 s


**Show word_count from vectorizer for products.iloc[269] (which is ... abracadabra .iloc[197]!!)**

In [31]:
train_data.iloc[197]["word_count"]

"{'A': 1, 'house': 1, 'in': 1, 'favorite': 1, 'our': 1}"

In [32]:
wc = sparse_vector_to_word_count(vectorizer.get_feature_names(),train_matrix[197])
wc

{'a': 1, 'favorite': 1, 'house': 1, 'in': 1, 'our': 1}

## Train the logistic regression classifier

In [33]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [34]:
sentiment_model = LogisticRegression()

Training on the ... **training data!**

In [35]:
%%time
sentiment_model.fit(train_matrix, y_train)

Wall time: 1min 15s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an Numpy array as follows:

**Adding the intercept to other coefficient!** (i.e. insert at index '0')

Also, rearrange in **row** vector

In [56]:
weights = np.insert(sentiment_model.coef_,0,sentiment_model.intercept_).reshape(1,-1)
weights.shape

(1, 121713)

There are a total of **121713** coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [57]:
num_positive_weights = (weights >= 0).sum()
num_negative_weights = (weights < 0).sum()
print("Number of positive weights:",num_positive_weights)
print("Number of negative weights:",num_negative_weights)

Number of positive weights: 85710
Number of negative weights: 36003


**Quiz question:** How many weights are >= 0?

In [58]:
print("Number of positive weights:",num_positive_weights)

Number of positive weights: 85710


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [41]:
sample_test_data = test_data[10:13]
print(sample_test_data["rating"])
sample_test_data

53    5
64    2
82    1
Name: rating, dtype: int64


,name,review,rating,review_clean,word_count,sentiment
53,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,"{'thrilled': 1, 'Scripture': 1, 'in': 1, 'Abso...",1
64,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,"{'as': 1, 'stick': 1, 'stuck': 1, 'applying': ...",-1
82,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,"{'wouldnt': 1, 'wallvery': 1, 'When': 1, 'exci...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [42]:
sample_test_data.iloc[0]["review"]

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [43]:
sample_test_data.iloc[1]["review"]

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

### Make predictions on this 3 samples

First, transform our sample using vectorizer to a sparse matrix

In [44]:
sample_test_matrix = vectorizer.transform(sample_test_data["review_clean"])

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

#### Computing scores
Using **sciki-learn** (*decision_function*):

In [86]:
scores = sentiment_model.decision_function(sample_test_matrix)
print("scores:", scores)

scores: [  5.5941264   -3.13116055 -10.40511896]


#### Computing scores
Using **my own function** (Computing the matrix!):

In [99]:
# Assuming:
# coef: row vector
# matrix: NxD sparse matrix where N=nb observations, D=nb of features

def compute_scores(coef,matrix):
    
    # transform matrix in array (not sparse!)
    matrix = matrix.toarray()
    
    # add feature x0 (i.e. a ones column vector)!
    matrix = np.insert(matrix,0,np.ones([1,matrix.shape[0]]),axis=1)
    
    scores = np.dot(matrix, np.transpose(coef))
    return(scores.reshape(1,-1))

In [100]:
my_scores = compute_scores(weights,sample_test_matrix)
print("my_scores:",my_scores)

my_scores: [[  5.5941264   -3.13116055 -10.40511896]]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

#### Predicting sentiment
Using **sciki-learn** (*predict*):

In [89]:
y_predict = sentiment_model.predict(sample_test_matrix)
print("sentiment prediction:", y_predict)

sentiment prediction: [ 1 -1 -1]


#### Predicting sentiment
Using **my own function** (From previously computed scores!):

In [90]:
def compute_predict(scores):
    scores[scores>0] = 1
    scores[scores<=0] = -1
    return(scores)

In [91]:
my_y_predict =  compute_predict(my_scores)
print("my_y_predict:",my_y_predict)

my_y_predict: [[ 1. -1. -1.]]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

#### Predicting probability sentiment
Using **sciki-learn** (*predict_proba*):

In [47]:
y_predict_proba = sentiment_model.predict_proba(sample_test_matrix)

In [49]:
print("sentiment probability prediction:")
print(y_predict_proba)

sentiment probability prediction:
[[  3.70586292e-03   9.96294137e-01]
 [  9.58159944e-01   4.18400565e-02]
 [  9.99969724e-01   3.02761816e-05]]


#### Predicting probability sentiment
Using **my own function** (From previously computed scores!):

In [101]:
def compute_predict_proba(scores):
    proba = 1/(1+np.exp(-scores))
    return(proba)

In [102]:
my_y_predict_proba = compute_predict_proba(my_scores)
print(my_y_predict_proba)

[[  9.96294137e-01   4.18400565e-02   3.02761816e-05]]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

**The third!**: 3.02761816e-05

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [ ]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    ...
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    ...

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    ...
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [ ]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [ ]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [ ]:
len(significant_words)

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [ ]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [ ]:
train_data[0]['review']

The **word_count** column had been working with before looks like the following:

In [ ]:
print train_data[0]['word_count']

Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [ ]:
print train_data[0]['word_count_subset']

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [ ]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [ ]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [ ]:
simple_model.coefficients

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [ ]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [ ]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?